# Compile the model

In [1]:
from keras import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers

Using TensorFlow backend.
/home/niraj/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model = Sequential()
model.add(Flatten(input_shape=(7,7,512)))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [3]:
adam = optimizers.Adam(lr=1e-6)

In [4]:
model.compile(optimizer= adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Import features from bcolz store

In [5]:
import bcolz

In [6]:
train_features = bcolz.open(rootdir='train_store', mode='r')
val_features = bcolz.open(rootdir='val_store', mode='r')

In [7]:
train_features.shape

(23000, 7, 7, 512)

In [8]:
val_features.shape

(2000, 7, 7, 512)

# Import labels from pickle file

In [1]:
import pickle

In [2]:
with open("train_labels.pkl", "rb") as f:
    train_labels = pickle.load(f)
f.close()

with open("val_labels.pkl", "rb") as f:
    val_labels = pickle.load(f)
f.close()

In [4]:
train_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [5]:
val_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

# Initialize keras callbacks

In [12]:
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, EarlyStopping

In [13]:
tb = TensorBoard(log_dir='./logs', batch_size=64, write_images=True, write_graph=True)

In [14]:
mc = ModelCheckpoint('./models/weights.{epoch:02d}-{val_loss:.3f}.hdf5', save_best_only=True)

In [15]:
cl = CSVLogger('./CSV/baseline.csv', separator=',')

In [16]:
es = EarlyStopping(monitor='val_loss', patience=5)

# Fit the model to the features

In [17]:
model.fit(train_features, 
          train_labels, 
          batch_size=64, 
          epochs=10, 
          callbacks = [tb, mc, cl, es],
          validation_data=(val_features, val_labels),
         )

Train on 23000 samples, validate on 2000 samples
Epoch 1/10
23000/23000 [==============================] - 27s 1ms/step - loss: 3.5821 - acc: 0.5025 - val_loss: 3.6355 - val_acc: 0.4945
Epoch 2/10
23000/23000 [==============================] - 26s 1ms/step - loss: 2.3209 - acc: 0.5920 - val_loss: 3.3991 - val_acc: 0.4885
Epoch 3/10
23000/23000 [==============================] - 26s 1ms/step - loss: 1.5152 - acc: 0.6909 - val_loss: 3.3193 - val_acc: 0.4860
Epoch 4/10
23000/23000 [==============================] - 26s 1ms/step - loss: 0.9913 - acc: 0.7918 - val_loss: 3.2140 - val_acc: 0.4905
Epoch 5/10
22208/23000 [===========================>..] - ETA: 0s - loss: 0.6593 - acc: 0.8737

KeyboardInterrupt: 

# TODO

* Try data augmentation
* Try adding dropout and batch normalization
* L2 Regualizer maybe?
* Submit